In [3]:
"""
I set the cropping to "loose" in pyannote.

"""


from tqdm import tqdm
import torch
from pathlib import Path
import numpy as np
from tqdm import tqdm
import pandas as pd
from pyannote.audio.features import Pretrained
from pyannote.core.utils.distance import l2_normalize
from pyannote.core.utils.distance import cdist
import os
import pickle
import librosa

def get_optimal_balanced_threshold(df):
    rows = []
    for i in np.arange(0, 1, 0.001):
        result = (df["dist"] < i).value_counts()
        if len(result) == 1:
            # If only True or False then its a bad threshold and move on
            continue
        ratio = result.loc[True] / result.loc[False]
        minimise_metric = abs((ratio) -1)
        rows.append({"metric": minimise_metric, "threshold":i, "ratio": ratio})
    _ = pd.DataFrame(rows)
    return _.sort_values("metric").iloc[0]

os.environ["PYANNOTE_DATABASE_CONFIG"] = "/media/ben/datadrive/Software/pyannote-audio/data/ami/"

expt_name = "juan_3"
emb = Pretrained(validate_dir='/media/ben/datadrive/Software/pyannote-audio/data/ami/voxceleb_juan_duration3/train/ZALODATASET.SpeakerVerification.BenProtocol.train/validate_equal_error_rate/ZALODATASET.SpeakerVerification.BenProtocol.development/',
                 epoch=15,
                 device="cuda")



/media/ben/datadrive/Software/pyannote-audio/data/ami/voxceleb_juan_duration3/config.yml
CONFIG {'duration': 0.75, 'label_min_duration': 10, 'margin': 0.05, 'per_epoch': 1, 'per_fold': 512, 'per_label': 1, 's': 10}
Loading weight /media/ben/datadrive/Software/pyannote-audio/data/ami/voxceleb_juan_duration3/train/ZALODATASET.SpeakerVerification.BenProtocol.train/weights/0015.pt


In [52]:

# speaker embedding model trained on AMI training set
# emb = torch.hub.load('pyannote/pyannote-audio', 'emb_voxceleb')
expt_root = "/media/ben/datadrive/Zalo/voice-verification/"
df_val = pd.read_csv("df_val.csv")

filename2embedding = {}
#with open(f"{expt_name}embedding_public.pickle", "rb") as input_file:
#    filename2embedding = pickle.load(input_file)


with tqdm(total=len(df_val)) as pbar:
    for i, row in df_val.iterrows():
        audio1_filepath = str(row["path"])
        if audio1_filepath in filename2embedding:
            audio1_embedding = filename2embedding[audio1_filepath]
        else:

            audio1_embedding = np.mean(emb(
                {"uri": row["path"], "audio": audio1_filepath}),
                                       axis=0, keepdims=True)
            filename2embedding[audio1_filepath] = audio1_embedding

        audio2_filepath = str(row["audio2_path"])
        if audio2_filepath in filename2embedding:
            audio2_embedding = filename2embedding[audio2_filepath]
        else:
            audio2_embedding = np.mean(emb(
            {"uri": row["audio2_path"], "audio": audio2_filepath}),
                                   axis=0, keepdims=True)
            filename2embedding[audio2_filepath] = audio2_embedding

        # X_audio1 = l2_normalize(np.array([audio1_embedding,]))
        # X_audio2 = l2_normalize(np.array([audio2_embedding,]))
        distance = cdist(audio1_embedding, audio2_embedding, metric="cosine")
        #if (i % 1000) == 0:
        #    print(f"Distance is {distance[0][0]} for index {i} ")
        dist = distance[0][0]
        df_val.loc[i, "dist"] = dist
        df_val.loc[i, "audio1_filepath"] = audio1_filepath
        df_val.loc[i, "audio2_filepath"] = audio2_filepath

        pbar.update()

with open(f"{expt_name}embedding_public.pickle", 'wb') as handle:
    pickle.dump(filename2embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_val.to_csv(f"{expt_name}_df_val.csv", index=False)

result = get_optimal_balanced_threshold(df_val)
print(result)
df_val["label"] = (df_val["dist"] < result["threshold"]).astype(int)

#df_write = df_val[["audio_1", "audio_2", "label"]]
#df_write.to_csv(f"{expt_name}_threshold{result['threshold']}.csv", index=False)


100%|██████████| 990/990 [01:53<00:00,  8.69it/s]


metric       0.004032
threshold    0.789000
ratio        0.995968
Name: 788, dtype: float64


In [54]:

chunks = emb.sliding_window
print(f'Embeddings were extracted every {1000 * chunks.step:g}ms on {1000 * chunks.duration:g}ms-long windows.')

Embeddings were extracted every 187.5ms on 750ms-long windows.


In [60]:
failure_cases = df_val[df_val["y_comparison"] != df_val["label"]]
import random

import IPython.display as ipd
index = random.randint(0,len(failure_cases))
print(f"For index: {index} predicted {failure_cases.iloc[index]['label']} incorrectly. {failure_cases.iloc[index]['uri']} VS {failure_cases.iloc[index]['audio2_uri']}  ")
ipd.Audio(failure_cases.iloc[index]["path"]) # load a local


For index: 37 predicted 1 incorrectly. 806-M-32_806-23 VS 512-M-34_512-31  


In [61]:
ipd.Audio(failure_cases.iloc[index]["audio2_path"]) # load a local

In [56]:
failure_cases.iloc[index]["path"], failure_cases.iloc[index]["audio2_path"]

('/media/ben/datadrive/Zalo/voice-verification/Train-Test-Data/dataset/454-M-25/454-48.wav',
 '/media/ben/datadrive/Zalo/voice-verification/Train-Test-Data/dataset/748-M-29/748-11.wav')

In [5]:
from sklearn.metrics import accuracy_score
accuracy_score(df_val["y_comparison"], df_val["label"])

0.9151515151515152

In [6]:

# speaker embedding model trained on AMI training set
# emb = torch.hub.load('pyannote/pyannote-audio', 'emb_voxceleb')
expt_root = "/media/ben/datadrive/Zalo/voice-verification/"
dataset_path = os.path.abspath(os.path.join(expt_root, "Train-Test-Data/public-test.csv"))
df_test_sub = pd.read_csv(dataset_path)

filename2embedding = {}
#with open(f"{expt_name}embedding_public.pickle", "rb") as input_file:
#    filename2embedding = pickle.load(input_file)


with tqdm(total=len(df_test_sub)) as pbar:
    for i, row in df_test_sub.iterrows():
        audio1_filepath = os.path.join(expt_root, "Train-Test-Data/public-test/") + str(row["audio_1"])
        if audio1_filepath in filename2embedding:
            audio1_embedding = filename2embedding[audio1_filepath]
        else:

            audio1_embedding = np.mean(emb(
                {"uri": row["audio_1"], "audio": audio1_filepath}),
                                       axis=0, keepdims=True)
            filename2embedding[audio1_filepath] = audio1_embedding

        audio2_filepath = os.path.join(expt_root, "Train-Test-Data/public-test/") + str(row["audio_2"])
        if audio2_filepath in filename2embedding:
            audio2_embedding = filename2embedding[audio2_filepath]
        else:
            audio2_embedding = np.mean(emb(
            {"uri": row["audio_2"], "audio": audio2_filepath}),
                                   axis=0, keepdims=True)
            filename2embedding[audio2_filepath] = audio2_embedding

        # X_audio1 = l2_normalize(np.array([audio1_embedding,]))
        # X_audio2 = l2_normalize(np.array([audio2_embedding,]))
        distance = cdist(audio1_embedding, audio2_embedding, metric="cosine")
        #if (i % 1000) == 0:
        #    print(f"Distance is {distance[0][0]} for index {i} ")
        dist = distance[0][0]
        df_test_sub.loc[i, "dist"] = dist
        df_test_sub.loc[i, "audio1_filepath"] = audio1_filepath
        df_test_sub.loc[i, "audio2_filepath"] = audio2_filepath


        pbar.update()

with open(f"{expt_name}embedding_public.pickle", 'wb') as handle:
    pickle.dump(filename2embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_test_sub.to_csv(f"{expt_name}_df_test_sub.csv", index=False)

result = get_optimal_balanced_threshold(df_test_sub)
print(result)
df_test_sub["label"] = (df_test_sub["dist"] < result["threshold"]).astype(int)


df_write = df_test_sub[["audio_1", "audio_2", "label"]]
df_write.to_csv(f"{expt_name}_threshold{result['threshold']}.csv", index=False)


100%|██████████| 50000/50000 [03:46<00:00, 220.76it/s]


metric       0.0004
threshold    0.7430
ratio        1.0004
Name: 726, dtype: float64
